In [1]:
#import torch
import zarr
import fsspec
import xarray as xr
#from torch import nn
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [14]:
model = NeuralNetwork().to('cuda')

RuntimeError: Invalid device string: 'cuda[1]'

In [15]:
model(torch.randn((784,)).to('cuda'))

tensor([ 0.0078,  0.1346,  0.0416,  0.0074, -0.1058, -0.1397, -0.2314,  0.1663,
         0.0029,  0.1660], device='cuda:0', grad_fn=<AddBackward0>)

In [29]:
z = zarr.zeros((10000, 10000), chunks=(1000, 1000), dtype='i4')

In [31]:
z.info

Type,zarr.core.Array
Data type,int32
Shape,"(10000, 10000)"
Chunk shape,"(1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.KVStore
No. bytes,400000000 (381.5M)
No. bytes stored,345
Storage ratio,1159420.3


In [2]:
ds = xr.open_zarr(fsspec.get_mapper('https://s3.bgc-jena.mpg.de:9000/xaida/SPEICube.zarr'), consolidated=True)

In [81]:
ds['spei_90'][:2,0,0].values

array([1.2908636, 1.2159431], dtype=float32)

In [6]:
spi = xr.open_dataset(Path('/scistor/ivm/data_catalogue/observations/EOBS/spi3_mon_0.1deg_reg_ens_median_E-OBSv23.1e.nc'))
spi = spi.drop('time_bnds')

In [7]:
spi['spi3ETCCDI'] = spi['spi3ETCCDI'].chunk({'latitude':50,'longitude':50})
temp = spi.to_zarr('/scistor/ivm/data_catalogue/observations/EOBS/test.zarr', compute = False)

In [ ]:
temp = spi.to_zarr('/scistor/ivm/data_catalogue/observations/EOBS/test.zarr',mode = 'a')